In [1]:
# load package
# string 
import re

# math
import pandas as pd
import numpy as np
import scipy as sp
import random

# sys
import sys
import os
import time
import warnings

# machine learning
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, gaussian_process, discriminant_analysis
from xgboost import XGBClassifier

# model utils
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection 
from sklearn import model_selection
from sklearn import metrics

# plot
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix #??
# = show plots in Jupyter Notebook browser
%matplotlib inline 
mpl.style.use('ggplot') #??
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8 #??

# show all columns
from IPython.display import display
pd.options.display.max_columns = None

# self define
sys.path.append('../../utils/')

In [115]:
# read files
lb_wsp_2014 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2014.csv', sep = ';')
lb_wsp_2015 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2015.csv', sep = ';')
lb_wsp_2016 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2016.csv', sep = ';')
lb_wsp_2017 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2017.csv', sep = ';')
lb_wsp_2018 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2018.csv', sep = ';')
std_wsp_2014 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2014.csv', sep = ';')
std_wsp_2015 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2015.csv', sep = ';')
std_wsp_2016 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2016.csv', sep = ';')
std_wsp_2017 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2017.csv', sep = ';')
std_wsp_2018 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2018.csv', sep = ';')
# cat
d1 = lb_wsp_2014.copy()
d2 = lb_wsp_2015.copy()
d3 = lb_wsp_2016.copy()
d4 = lb_wsp_2017.copy()
d5 = lb_wsp_2018.copy()
data1 = pd.concat([d1,d2,d3,d4], 0)
d6 = std_wsp_2014.copy()
d7 = std_wsp_2015.copy()
d8 = std_wsp_2016.copy()
d9 = std_wsp_2017.copy()
d10 = std_wsp_2018.copy()
data2 = pd.concat([d6,d7,d8,d9,d10], 0)
data = pd.concat([data1, data2], 0)

/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## AW-Nr

In [5]:
aw = data['AW-Nr']

In [6]:
aw.describe()

count      1461205
unique       31639
top       99999994
freq         64046
Name: AW-Nr, dtype: object

In [33]:
aw_new = aw.map(lambda x: re.sub('[a-zA-Z]', '-', str(x)))

In [22]:
aw_new.head()

0              
1              
2    51017133.0
3    51017195.0
4    51799999.0
Name: AW-Nr, dtype: object

## Teile-Nr

## Beschreibung
操作包括
- 全体大写
- 缩写回复
- 去标点
- 特殊字母回复
- 去除数值
- 去除单字母的单词
- 单复数转换

In [117]:
bs = data['Beschreibung']
bs.describe()

count      3175588
unique      152879
top       SCHRAUBE
freq        261756
Name: Beschreibung, dtype: object

In [181]:
# operate
from pattern.de import parse, conjugate, singularize, pluralize

def bs_operate(li):
    """
    input string:
        'describe' attribute 
    """
    # to string
    li = str(li).lower()
    # recover: problem with '.' in string
    #dict_re = {'ae': 'ä', 'oe': 'ö', 'ue': 'ü', 'ß': 'ss', 'u.': 'und ', 'fzg': 'fahrzeug', ' f ': ' für '}
    dict_re = {'ae': 'ä', 'oe': 'ö', 'ue': 'ü', 'ß': 'ss', 'fzg': 'fahrzeug', ' f ': ' '}
    for i, j in dict_re.items():
        if i in li:
            li = re.sub(i, j, li)
    #print(li,0)
    # replace punctuation with ' '
    li = re.sub("[\s+\.\!_,$%^*(+\"\')-:]", " ", li)
    #print(li,1)
    # remove the number in the string
    li = re.sub('\d', '', li)
    #print(li,2)
    # remove word with one ziffer
    li = re.sub(' [a-zA-Z] ', "", li)
    #print(li,3)
    # plural to odd
    words = li.strip().split(' ')
    words_sin = [pluralize(word.strip()) if len(word) > 0 and parse(word).split('/')[1] == 'NN' and parse(word).split('/')[1] != 'IN' else word.strip() for word in words]
    #words_sin = [pluralize(word.strip()) if parse(word).split('/')[1] == 'NN' for word in words]
    li = ' '.join(words_sin)
    #print(li,4)
    # upper
    li = li.upper()
    #print(li,5)
    return li

In [1]:
#ab['Beschreibung2'] = ab['Beschreibung'].map(bs_operate)

In [190]:
ab['Beschreibung'] = ab['Beschreibung'].map(lambda x: re.sub('[.,/-]', ' ', str(x).upper().strip()).strip())

/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


## + AW-Nr
- 属性互补

In [101]:
ab = lb_wsp_2014[['AW-Nr', 'Beschreibung']]
ab['AW-Nr'] = ab['AW-Nr'].map(lambda x: str(x))

/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [192]:
ab.sort_values(['AW-Nr'], inplace = True)
ab = ab.reset_index()

/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [193]:
#ab.loc[1342100:1342150]
ab = ab[['AW-Nr','Beschreibung']]
ab.head()

,AW-Nr,Beschreibung
0,1000099.0,hol und bring service
1,1000099.0,hol und bring service
2,1000099.0,hol und bring service
3,10010199.0,prüfvoraussetzungen lt diagnose herstellen
4,10010199.0,bennräume endoskopiert


In [117]:
fake = data['AW-Nr'].map(lambda x: bool(re.search('[a-zA-Z]', str(x))))

In [122]:
data[fake]['AW-Nr'].describe()

count     7968
unique    1041
top       LACK
freq      1553
Name: AW-Nr, dtype: object

In [123]:
data[fake]['AW-Nr'].unique()

array([nan, 'PX955500', 'PXISI710', ..., 'E130100', 'W+I', 'J46240008'], dtype=object)

In [194]:
tmp = complementary(ab)

6556
12664
hol und bring service ['1000099.0']
prüfvoraussetzungen lt diagnose herstellen ['10010199.0']
False


- 两个属性，内容互补

In [145]:
# 初衷是，A的种类比B的种类要少很多， 想通过A，来对B进行分类，但是A存在大量的空值，所以想通过B，把对应的A的值补齐
def complementary(df):
    # assert: df have two columns
    #TODO
    
    # if no null value return df
    #TODO
    
    headers = df.columns.values.tolist()
    # save the data to tmp dir and read again with txt type
    df.to_csv('/tmp/tmp.csv', sep = ';', index = False)
    dic = ABDict('/tmp/tmp.csv', header = True, sep = ';')
    #df[headers[1]] = df.apply(_comAB, axis = 1, args = [dic])
    df[headers[0]] = df.apply(_comBA, axis = 1, args = [dic])
    return df

def _comAB(x, dic):
    if str(x[1]) == 'nan':
        return dic.AToB(x[0])
    else:
        return x[1]

def _comBA(x, dic):
    if str(x[0]) == 'nan':
        return dic.BToA(x[1])
    else:
        return x[0]
    
class ABDict:
    """
    make sure that only two columns in the df, otherwise only the first two columns will be used
    用于两列数据互补空值
    """
    def __init__(self, path, header = True, sep = ';'):
        self.dic = self._load_dict(path, header, sep) 
        self.dic_conv = self._load_dict_conv(path, header, sep)
        print(len(self.dic))
        print(len(self.dic_conv))
        counter = 0
        ########
        # test #
        ########
        #for i in self.dic.keys():
        #    if counter > 1:
        #        break
        #    print(i, self.dic[i])
        #    counter += 1
        #print('schraube' in self.dic_conv.keys())
    
    def AToB(self, a):
        if a in self.dic.keys():
            out = self.dic[a][0]
            if type(out) == list:
                print(out)
            return out
        else:
            #print(a)
            return float('nan')
    
    def BToA(self, b):
        if b in self.dic_conv.keys():
            out = self.dic_conv[b][0]
            #print(out)
            return out
        else:
            #print(b)
            return float('nan')

    def _load_dict(self, path, header, sep):
        dic = {} 
        with open(path) as fi:
            counter = 0
            for li in fi:
                if header and counter ==0:
                    # drop first line if header is true
                    counter += 1
                    continue
                li = li.strip()   # 忘记去换行符了，导致调了一晚上的错
                items = li.split(sep)
                if items[0] not in dic.keys():
                    # if not exit, add new item
                    if items[1] != 'nan':
                        dic[items[0]] = [items[1]]
                elif items[1] not in dic[items[0]]:
                    # if item exit but value not eixt, add the value to the list
                    dic[items[0]].append(items[1])
                counter += 1
        return dic
    
    def _load_dict_conv(self, path, header, sep):
        dic = {}
        with open(path) as fi:
            counter = 0
            for li in fi:
                if header and counter == 0:
                    # drop first line if header is true
                    counter += 1
                    continue
                li = li.strip()
                items = li.split(sep)
                if items[1] not in dic.keys():
                    if items[0] != 'nan':
                        #print(items[0])
                        dic[items[1]] = [items[0]]
                elif items[0] not in dic[items[1]]:
                    dic[items[1]].append(items[0])
                counter += 1
        return dic

In [71]:
# 初衷是，根据单词出现的次数进行分类，
# 字典是用于获取对每个单词的出现次数进行排序，并提取排名前几位的单词
# 提取并视察前几名的单词，试着把他用到beschreibung属性里面
from pattern.de import parse, conjugate, singularize, pluralize

#headers = ab.columns.values.tolist()
# save the data to tmp dir and read again with txt type
#ab['Beschreibung'].to_csv('/tmp/tmp.csv', sep = ';', index = False)
#dic = FreDict('/tmp/tmp.csv', header = True, sep = ';')
#df[headers[1]] = df.apply(_comAB, axis = 1, args = [dic])
#dic.top(50)
#dic.get_count('mit')

class FreDict:
    """
    统计单词和出现的频率
    文档中的每个字都会被记录进去，所以应该确认输入的文件内容，只含有目标列属性
    """
    def __init__(self, path, header = True, sep = ';', clean = False, recover = False, singular = False):
        """
        input:
            path: string
                the path of the file
            header: boolean
                weather the file contains header of not
            sep: string
                the sep of the file in each line
            clean: boolean
                remove the word with single buchstachben or not
            recover: boolean
                transform 'ae', 'oe', 'ue', 'ss' back to 'ä', 'ö', 'ü', 'ß' usw.
            singular: boolean
                weather turns the word to singular or not
        """
        self.dict_re = {'ae': 'ä', 'oe': 'ö', 'ue': 'ü', 'ß': 'ss', 'u.': 'und ', 'fzg': 'fahrzeug', ' f ': ' für '}
        self.words, self.dict_count = self._load_data(path, header, sep, clean, recover, singular) # list of Words
        self.ls_sorted = self._sort_dict(self.words) # list of list
        self.len = len(self.words)
    
    def get_word(self, w):
        if w in self.dict_count.keys():
            return self.dict_count[w]
        else:
            return Word('x', 0, 'NAW')
    
    def get_best(self, s):
        """
        input:
            s string:
                list of words in form of string
        output:
            out string:
                the frequentest word
        """
        words = s.split(' ')
        words = [word.strip() for word in words]
        out = 'xxx'
        count = 0
        for word in words:
            tmp = self.get_word(word)
            if tmp.get_count() > count:
                out = tmp.get_word()
                count = tmp.get_count()
        return out
        
    def top(self, n = 10):
        # return list of list
        # get the top n item 
        if n == -1:
            return self.ls_sorted
        return self.ls_sorted[:n]
    
    def top_norm(self, n = 10):
        # get the top n norm
        out = []
        counter = 0
        for i in self.ls_sorted:
            if counter >= n and n != -1:
                break
            if i[2] == 'NN' or i[2] == 'NNP' or i[2] == 'NNS' or i[2] == 'NNPS':
                out.append(i)
            counter += 1
        return out
    
    def top_verb(self, n = 10):
        # get the top n verb
        out = []
        counter = 0
        for i in self.ls_sorted:
            if counter >= n and n != -1:
                break
            if i[2] == 'VB' or i[2] == 'VBZ' or i[2] == 'VBP' or i[2] == 'VBD' or i[2] == 'VBN' or i[2] == 'VBG':
                out.append(i)
            counter += 1
        return out
    
    def _load_data(self, path, header, sep, clean, recover, singular):
        """
        return list of words and count dictionary
        """
        dic = {}
        with open(path) as fi:
            counter = 0
            for li in fi:
                if header and counter == 0:
                    # drop first line if header is true
                    counter += 1
                    continue
                li = li.strip()
                if recover:
                    # recover same express if necessary
                    for i, j in self.dict_re.items():
                        if i in li:
                            li = re.sub(i, j, li)
                items = re.split('[;, ]', li)
                #items = li.split(sep)
                for item in items:
                    if item not in dic.keys():
                        if clean and len(item) <= 1:
                            # remove the item with less than 2 ziffer
                            continue
                        else:
                            dic[item] = 1
                    else:
                        dic[item] += 1
                counter += 1
        # transform to Word
        words = []
        for i, j in dic.items():
            try:
                if singular:
                    i = singularize(i).lower()
                words.append(Word(i, j, parse(i).split('/')[1]))
                #words.append(Word(i, j, parse(conjugate(i)).split('/')[1]))
            except:
                words.append(Word(i, j, ''))
        # pack the dic
        dic = {i: Word(i, j, parse(i).split('/')[1]) for i, j in dic.items() if len(i) > 0 }
        # return 
        return words, dic
    
    def _sort_dict(self, words):
        # return list of words, sorted according to the count
        li_sorted = sorted(words, key = lambda x: x.get_count(), reverse = True) 
        # transform to list
        out = []
        for i in li_sorted:
            out.append(i.to_list())
        return out
    
class Word:
    """
    use @property to rewrite the class
    """
    def __init__(self, word = 'NN', count = 1, tag = 'NN'):
        self.word = word
        self.count = count
        self.tag = tag

    def to_list(self):
        out = [self.word, self.count, self.tag]
        return out
    
    def __str__(self):
        return self.word + " " + self.count + " " + self.tag

    def get_count(self):
        return self.count

    def get_tag(self):
        return self.tag

    def get_word(self):
        return self.word

    def set_count(self, c):
        self.count = c

    def set_tag(self, t):
        self.tag = t

    def set_word(self, w):
        self.word = w

,AW-Nr,Beschreibung


In [193]:
headers = ab.columns.values.tolist()
# save the data to tmp dir and read again with txt type
ab['Beschreibung'].to_csv('/tmp/tmp.csv', sep = ';', index = False) # 此sep和下面的是不同的，
dic = FreDict('/tmp/tmp.csv', header = True, sep = ';', clean = True, recover = True, singular = True)
#df[headers[1]] = df.apply(_comAB, axis = 1, args = [dic])
#dic.get_count('mit')

In [195]:
dic.top(100)

[['und', 27983, 'CC'],
 ['aund', 23578, 'NN'],
 ['schraub', 21697, 'NN'],
 ['eingebaund', 21466, 'NN'],
 ['aus-', 7172, 'IN'],
 ['für', 6133, 'IN'],
 ['u.eingebaut', 5396, 'NN'],
 ['filterein', 5235, 'NN'],
 ['servic', 5150, 'FW'],
 ['dichtring', 4727, 'FW'],
 ['wischblatt', 4568, 'NN'],
 ['vor', 4496, 'IN'],
 ['ersetzt', 4368, 'VBN'],
 ['hinte', 3961, 'NN'],
 ['longlif', 3955, 'NN'],
 ['iii', 3719, 'NN'],
 ['pollenfilter', 3526, 'NN'],
 ['staund', 3525, 'NN'],
 ['wert', 3469, 'RP'],
 ['räd', 3332, 'NN'],
 ['-und', 3202, '-'],
 ['vo', 3157, 'NN'],
 ['fzg.-wäsch', 3153, 'NN'],
 ['im', 3143, 'IN'],
 ['wert', 3130, 'RP'],
 ['-euro', 3127, '-'],
 ['5w30', 3096, 'NN'],
 ['gfs/geführt', 3030, 'NN'],
 ['funktion', 3002, 'NN'],
 ['a+', 2967, 'PRP'],
 ['dichtung', 2918, 'NN'],
 ['im', 2867, 'IN'],
 ['vo', 2808, 'NN'],
 ['fahrzeugwäsch', 2794, 'NN'],
 ['staubfilt.', 2740, 'NN'],
 ['00\x80', 2733, 'NN'],
 ['vor', 2701, 'IN'],
 ['und', 2541, 'CC'],
 ['scheibenreiniger', 2450, 'NNP'],
 ['gfs/geführ

## Adressanredecode

In [350]:
av  = lb_wsp_2014[['Auftragsdatum', 'Adressanredecode']]

In [365]:
av_ref = toFreqTable(av, 'Adressanredecode', 'Auftragsdatum')

Fake date exist, drop these date directly


In [367]:
av_ref.head(10)

,Auftragsdatum,Adressanredecode,count
0,2013-12-30,Firma,2.0
1,2014-01-02,Firma,95.0
2,2014-01-02,Frau,14.0
3,2014-01-02,Herr,19.0
4,2014-01-03,Firma,4.0
5,2014-01-03,Herr,74.0
6,2014-01-03,OHNE,2.0
7,2014-01-04,AN,15.0
8,2014-01-04,FA_F,22.0
9,2014-01-04,Firma,525.0


In [366]:
# 初衷是通过一个个的账单，统计每个属性，对应属性项在一段时间内出现的次数

def toFreqTable(df, att, date, clean = True):
    """
    input:
        df, DataFrame:
            the dataframe
        att, string:
            the column name of the target attribute
        date, string:
            the column name of the date attribute
    """
    # extract the att and date columns
    df = df[[att, date]]
    
    # change string date to date type
    # detect outlier
    fake = df[date].map(check_date)
    # if clean is True, drop the fake data
    if clean:
        print("Fake date exist, drop these dates directly")
        df = df.drop(df[fake].index)
    df[date] = pd.to_datetime(df[date])
    # add new column named 'count'
    df['count'] = pd.Series([1 for i in range(len(df))])
    # group and sum 
    df_g = df.groupby([date, att], as_index = False).agg({'count': 'sum'})
    return df_g

def check_date(s, sep = '.'):
    """
    only check the date grobly,
    set the fake date to True and the right date to false
    """
    items = s.split('.')
    #print(items)
    if len(items) <3:
        return True
    day = int(items[0])
    month = int(items[1])
    year = int(items[2])
    if day <= 31 and day > 0 and month <= 12 and month > 0 and year > 0:
        return False
    else:
        return True

# Beschreibung2

In [13]:
ab  = lb_wsp_2014[['Auftragsnummer', 'Beschreibung']]

In [16]:
ab  = lb_wsp_2014[['Auftragsnummer', 'Beschreibung']]
tmp = toAuftragTable(ab, att = 'Beschreibung', auftn = 'Auftragsnummer', clean = True)

Null date exist, drop these dates directly


In [218]:
# 给的数据的每一行都是一个维修项，初衷是，把属于同一个auftrag的维修项合并到一起，看一下，在同一个Auftrag中，经常一起修的是那些内容

def toAuftragTable(df, att, auftn, clean = True):
    """
    input:
        df, DataFrame:
            the dataframe
        att, string:
            the column name of the target attribute
        auftn, string:
            the column name of the aftragsnummer attribute
    output:
        df_g, DataFrame:
            dataframe contrains two columns auftn and att
            type of item in att is string, separate with ';'
    """
    # assert: make sure the type of the attributes inputted
    
    # extract the att and date columns
    df = df[[att, auftn]]
    # if clean is True, drop the fake data, like the null data
    if clean:
        print("Null date exist, drop these dates directly")
        df = df.drop(df[df[att].isnull()].index)
        df = df.drop(df[df[auftn].isnull()].index)
    # group and sum 
    df_g = df.groupby([auftn], as_index = False).apply(agg)
    return df_g

# apply 只能对单行进行处理，而不是对整个分组进行处理，所以估计应该把axis换成1，比较好
def agg(x):
    # 是否用‘ ’分隔会比较好，这样就不用对初始的属性，
    x = x.apply(lambda x: ';'.join(set([str(i) for i in x])), axis = 0)
    #x = x.apply(lambda x: ' '.join(set(x)), axis = 0)
    #print(x.columns.values)
    return x

In [219]:
a = tmp['Beschreibung'][3]

KeyError: 'Beschreibung'

In [27]:
list(a)

['BASISLACK',
 'SEITENTEIL HINTEN AUSGEBEULT',
 'KAROSSERIE LACKIERUNG VORBEREITET',
 'TÜR HINTEN LACKIERT',
 'GFS/GEFÜHRTE FUNKTION .',
 'TÜRGRIFF AUS- U.EINGEBAUT',
 'ENTLÜFTUNGSBLENDE AUS- U.EINGEBAUT',
 'ABDECKUNG F STOßFÄNGER HINTEN A+E',
 'RAD AUS- U.EINGEBAUT',
 'SEITENTEIL HINTEN LACKIERT',
 'SCHEIBENRAD LACKIERT',
 'VERBRINGUNG KUNDENFAHRZEUG .',
 'SCHLUSSLEUCHTEN AUS- U.EINGEBAUT',
 'FAHRZEUG VORN U.HINTEN GEMESSEN',
 'TANKKLAPPE AUS- U.EINGEBAUT',
 'TÜRVERKLEIDUNG HINTEN AUS- U.EINGEBAUT',
 'KONSERVIER',
 'RADHAUSSCHALE HINTEN AUS- U.EINGEBAUT',
 'FENSTERSCHACHTABDICHTUNG HINTEN A+E',
 'Karosserie konserviert',
 'RAD GEWUCHTET',
 'SCHUTZMAT.',
 'TANKKLAPPE LACKIERT',
 'TÜR HINTEN AUS- U.EINGEBAUT',
 'REIFEN SICHTPRÜFEN',
 'SCHWEMMAT.',
 'FZG. REINIGEN ENTFERNUNG REP.-SPUREN',
 'SCHRIFTZUG " AH WEEBER" ANBRINGEN',
 'TÜR HINTEN AUSGEBEULT']

## Auftragsnummer

In [81]:
a = lb_wsp_2014['Auftragsnummer']

In [82]:
fake = a.map(lambda x: True if len(x) != 10 else False)

In [83]:
# 初衷是为了找出Auftragsnummer的异常项，目前的想法是默认这个nummer的长度是固定的，随意所有不在长度不同的，都会被归为异常
def check_auftragsnummer(x, leng = 10):
    if len(x) != 10:
        return True
    else:
        return False

In [84]:
tmp = a[fake]

In [86]:
aa = lb_wsp_2014[['Auftragsnummer', 'Auftragsdatum']]

In [100]:
lb_wsp_2014[fake][['Auftragsnummer', 'Teile-Nr', 'Auftragsdatum']]

,Auftragsnummer,Teile-Nr,Auftragsdatum
14067,103K,HDV,0
14069,103K,HDV,0
14071,103K,HDV,0
23457,103K,HDV,0
23459,103K,HDV,0
23461,103K,HDV,0
23463,103K,HDV,0
23465,103K,HDV,0
23467,103K,HDV,0
23469,103K,HDV,0


# FP-Growth

In [60]:
"""
A Python implementation of the FP-growth algorithm.
Basic usage of the module is very simple:
    > from fp_growth import find_frequent_itemsets
    > find_frequent_itemsets(transactions, minimum_support)
"""

from collections import defaultdict, namedtuple
try:
    from itertools import imap
except ImportError:
    # Python 3...
    imap=map

__author__ = 'Eric Naeseth <eric@naeseth.com>'
__copyright__ = 'Copyright © 2009 Eric Naeseth'
__license__ = 'MIT License'

def find_frequent_itemsets(transactions, minimum_support, include_support=False):
    """
    Find frequent itemsets in the given transactions using FP-growth. This
    function returns a generator instead of an eagerly-populated list of items.
    The `transactions` parameter can be any iterable of iterables of items.
    `minimum_support` should be an integer specifying the minimum number of
    occurrences of an itemset for it to be accepted.
    Each item must be hashable (i.e., it must be valid as a member of a
    dictionary or a set).
    If `include_support` is true, yield (itemset, support) pairs instead of
    just the itemsets.
    """
    print("what the fuck")
    items = defaultdict(lambda: 0) # mapping from items to their supports
    # Load the passed-in transactions and count the support that individual
    # items have.
    for transaction in transactions:
        for item in transaction:
            items[item] += 1
    #print(items)        
    # Remove infrequent items from the item support dictionary.
    items = dict((item, support) for item, support in items.items()
        if support >= minimum_support)
    #print(items)
    # Build our FP-tree. Before any transactions can be added to the tree, they
    # must be stripped of infrequent items and their surviving items must be
    # sorted in decreasing order of frequency.
    def clean_transaction(transaction):
        transaction = filter(lambda v: v in items, transaction)
        #transaction.sort(key=lambda v: items[v], reverse=True)
        transaction = sorted(transaction, key = lambda v: items[v], reverse = True)
        return transaction
    print("Build FPTree")
    print("num of transactions: "+str(len(transactions)))
    master = FPTree()
    for transaction in imap(clean_transaction, transactions):
        master.add(transaction)
    def find_with_suffix(tree, suffix):
        for item, nodes in tree.items():
            #print(item, nodes)
            support = sum(n.count for n in nodes)
            if support >= minimum_support and item not in suffix:
                # New winner!
                found_set = [item] + suffix
                yield (found_set, support) if include_support else found_set

                # Build a conditional tree and recursively search for frequent
                # itemsets within it.
                cond_tree = conditional_tree_from_paths(tree.prefix_paths(item))
                for s in find_with_suffix(cond_tree, found_set):
                    yield s # pass along the good news to our caller
                    
    # Search for frequent itemsets, and yield the results we find.
    for itemset in find_with_suffix(master, []):
        #print(itemset)
        yield itemset
        
class FPTree(object):
    """
    An FP tree.
    This object may only store transaction items that are hashable
    (i.e., all items must be valid as dictionary keys or set members).
    """
    
    Route = namedtuple('Route', 'head tail')
    
    def __init__(self):
        # The root node of the tree.
        self._root = FPNode(self, None, None)

        # A dictionary mapping items to the head and tail of a path of
        # "neighbors" that will hit every node containing that item.
        self._routes = {}
        
    @property
    def root(self):
        """The root node of the tree."""
        return self._root
    
    def add(self, transaction):
        """Add a transaction to the tree."""
        point = self._root
        
        for item in transaction:
            next_point = point.search(item)
            if next_point:
                # There is already a node in this tree for the current
                # transaction item; reuse it.
                next_point.increment()
            else:
                # Create a new point and add it as a child of the point we're
                # currently looking at.
                next_point = FPNode(self, item)
                point.add(next_point)

                # Update the route of nodes that contain this item to include
                # our new node.
                self._update_route(next_point)
                
            point = next_point
            
    def _update_route(self, point):
        """Add the given node to the route through all nodes for its item."""
        assert self is point.tree
        
        try:
            route = self._routes[point.item]
            route[1].neighbor = point # route[1] is the tail
            self._routes[point.item] = self.Route(route[0], point)
        except KeyError:
            # First node for this item; start a new route.
            self._routes[point.item] = self.Route(point, point)
            
    def items(self):
        """
        Generate one 2-tuples for each item represented in the tree. The first
        element of the tuple is the item itself, and the second element is a
        generator that will yield the nodes in the tree that belong to the item.
        """
        for item in self._routes.keys():
            yield (item, self.nodes(item))
            
    def nodes(self, item):
        """
        Generate the sequence of nodes that contain the given item.
        """
        try:
            node = self._routes[item][0]
        except KeyError:
            return
        while node:
            yield node
            node = node.neighbor
            
    def prefix_paths(self, item):
        """Generate the prefix paths that end with the given item."""
        def collect_path(node):
            path = []
            while node and not node.root:
                path.append(node)
                node = node.parent
            path.reverse()
            return path
        
        return (collect_path(node) for node in self.nodes(item))
    
    def inspect(self):
        print('Tree:')
        self.root.inspect(1)
        #print
        print('Routes:')
        for item, nodes in self.items():
            print('  %r' % item)
            for node in nodes:
                print('    %r' % node)
                
def conditional_tree_from_paths(paths):
    """Build a conditional FP-tree from the given prefix paths."""
    tree = FPTree()
    condition_item = None
    items = set()
    
    # Import the nodes in the paths into the new tree. Only the counts of the
    # leaf notes matter; the remaining counts will be reconstructed from the
    # leaf counts.
    for path in paths:
        if condition_item is None:
            condition_item = path[-1].item
            
        point = tree.root
        for node in path:
            next_point = point.search(node.item)
            if not next_point:
                # Add a new node to the tree.
                items.add(node.item)
                count = node.count if node.item == condition_item else 0
                next_point = FPNode(tree, node.item, count)
                point.add(next_point)
                tree._update_route(next_point)
            point = next_point
            
    assert condition_item is not None
    
    # Calculate the counts of the non-leaf nodes.
    for path in tree.prefix_paths(condition_item):
        count = path[-1].count
        for node in reversed(path[:-1]):
            node._count += count
            
    return tree

class FPNode(object):
    """A node in an FP tree."""
    def __init__(self, tree, item, count=1):
        self._tree = tree
        self._item = item
        self._count = count
        self._parent = None
        self._children = {}
        self._neighbor = None
        
    def add(self, child):
        """Add the given FPNode `child` as a child of this node."""
        if not isinstance(child, FPNode):
            raise TypeError("Can only add other FPNodes as children")
            
        if not child.item in self._children:
            self._children[child.item] = child
            child.parent = self

    def search(self, item):
        """
        Check whether this node contains a child node for the given item.
        If so, that node is returned; otherwise, `None` is returned.
        """
        try:
            return self._children[item]
        except KeyError:
            return None
        
    def __contains__(self, item):
        return item in self._children
    
    @property
    def tree(self):
        """The tree in which this node appears."""
        return self._tree
    
    @property
    def item(self):
        """The item contained in this node."""
        return self._item
    
    @property
    def count(self):
        """The count associated with this node's item."""
        return self._count
    
    def increment(self):
        """Increment the count associated with this node's item."""
        if self._count is None:
            raise ValueError("Root nodes have no associated count.")
        self._count += 1
        
    @property
    def root(self):
        """True if this node is the root of a tree; false if otherwise."""
        return self._item is None and self._count is None
    
    @property
    def leaf(self):
        """True if this node is a leaf in the tree; false if otherwise."""
        return len(self._children) == 0
    
    @property
    def parent(self):
        """The node's parent"""
        return self._parent
    
    @parent.setter
    def parent(self, value):
        if value is not None and not isinstance(value, FPNode):
            raise TypeError("A node must have an FPNode as a parent.")
        if value and value.tree is not self.tree:
            raise ValueError("Cannot have a parent from another tree.")
        self._parent = value
        
    @property
    def neighbor(self):
        """
        The node's neighbor; the one with the same value that is "to the right"
        of it in the tree.
        """
        return self._neighbor
    
    @neighbor.setter
    def neighbor(self, value):
        if value is not None and not isinstance(value, FPNode):
            raise TypeError("A node must have an FPNode as a neighbor.")
        if value and value.tree is not self.tree:
            raise ValueError("Cannot have a neighbor from another tree.")
        self._neighbor = value
        
    @property
    def children(self):
        """The nodes that are children of this node."""
        return tuple(self._children.itervalues())
    
    def inspect(self, depth=0):
        print(('  ' * depth) + repr(self))
        for child in self.children:
            child.inspect(depth + 1)
            
    def __repr__(self):
        if self.root:
            return "<%s (root)>" % type(self).__name__
        return "<%s %r (%r)>" % (type(self).__name__, self.item, self.count)

In [ ]:
result = []
for itemset, support in find_frequent_itemsets(transactions, 10, True):
    result.append((itemset,support))
result = sorted(result, key=lambda i: i[0])
for itemset, support in result:
    print(str(itemset) + ' ' + str(support))

- 如果是数字类型的，那么输入会是list of list
- 如果是字符串类型的，那么输入会是list of string，然而根据代码，这种情况是存在问题的
- 所以还是，别以pandas csv的方式读取吧， 直接读了，然后通过index dict转化为index，最后生成列表类，再输入到find_frequent_itemsets(transactions, options.minsup, True)方法就行了。

In [5]:
# 初衷是写一个dict，使得可以把string类型的对象，转化为index，并提供回复的方法
from pattern.de import singularize, pluralize

class IdxDict:
    """
    文件应该只含有一列, 字符串类型，描述Auftrag的内容, 或者是Auftrag，合并前的内容
    """
    def __init__(self, path, header = True, sep = ';'):
        #self.dict_re = {'ae': 'ä', 'oe': 'ö', 'ue': 'ü', 'ß': 'ss', 'u.': 'und ', 'fzg': 'fahrzeug', ' f ': ' für '}
        self.dict_to_index = self._load_dict(path)
        self.dict_to_word = {v:x for x, v in self.dict_to_index.items()}
        print(len(self.dict_to_index))
        print(len(self.dict_to_word))
    
    def inx_to_word(self, inx):
        if inx in self.dict_to_word.keys():
            return self.dict_to_word[inx]
        else:
            print('inputted index does not exit')
    
    def li_to_idx(self, li, sep = ';'):
        out = []
        li = str(li)
        items = li.split(sep)
        for item in items:
            #print(item)
            out.append(self.word_to_idx(item))
        return out
        
    def word_to_idx(self, word):
        if word in self.dict_to_index.keys():
            return self.dict_to_index[word]
        else:
            print('inputted word does not exit')
            return -1
    
    def _load_dict(self, path, header = True, sep = ';'):
        dic = {}
        with open(path) as fi:
            counter = 0
            index = 0
            for li in fi:
                if header and counter == 0:
                    # drop first line if header is true
                    counter += 1
                    continue
                li = li.strip()
                items = li.split(sep)
                for item in items:
                    item = item.strip("'")
                    if item not in dic.keys():
                        dic[item] = index
                        index += 1
                counter += 1
        return dic

## Beschreibung3
- 转化到Auftrag table
- 生成IdxDict
- 转化为Idx
- 做apriori

In [8]:
ab  = lb_wsp_2014[['Auftragsnummer', 'Beschreibung']]
tmp = toAuftragTable(ab, att = 'Beschreibung', auftn = 'Auftragsnummer', clean = True)

Null date exist, drop these dates directly


In [9]:
tmp['Beschreibung'].to_csv('/tmp/tmp.csv', sep = ';', index = False)

In [10]:
# 
idxDict = IdxDict('/tmp/tmp.csv', header = False)

21328
21328


In [12]:
#tmp['Beschreibung2'] = tmp['Beschreibung'].map(idxDict.li_to_idx)

In [13]:
tmp['Beschreibung2'].to_csv('/tmp/tmp.csv', sep = ';', index = False)

In [14]:
lis = [li.rstrip('\n')[1:-1] for li in open('/tmp/tmp.csv')]

In [15]:
lin = [li.split(',') for li in lis]

In [16]:
idx = []
for li in lin:
    tmp = []
    for i in li:
        tmp.append(int(i.strip()))
    idx.append(tmp)

In [31]:
result = []
for itemset, support in find_frequent_itemsets(idx, 10, True):
    result.append((itemset,support))
result = sorted(result, key=lambda i: i[0])
for itemset, support in result:
    print(str(itemset) + ' ' + str(support))

what the fuck
defaultdict(<function find_frequent_itemsets.<locals>.<lambda> at 0x1a14407ae8>, {0: 2, 1468: 26, 2: 1, 3: 1, 24: 1931, 6837: 22, 6: 534, 7: 143, 8: 113, 9: 87, 10: 11, 11: 4, 12: 214, 13: 180, 14: 49, 15: 23, 16: 13, -1: 3912, 18: 40, 19: 53, 20: 30, 21: 13, 22: 55, 23: 96, 25: 158, 26: 168, 27: 2, 28: 187, 29: 541, 30: 34, 31: 246, 32: 1, 294: 212, 532: 25, 35: 1028, 36: 304, 37: 23, 38: 3494, 39: 1392, 40: 195, 41: 2104, 42: 901, 43: 546, 44: 750, 45: 995, 46: 366, 47: 285, 48: 1222, 49: 6531, 50: 500, 51: 2222, 52: 429, 53: 1236, 54: 1935, 62: 1042, 57: 126, 58: 5, 59: 2523, 60: 1583, 61: 2833, 330: 1702, 65: 10, 66: 1, 67: 95, 68: 32, 69: 144, 70: 217, 71: 102, 72: 126, 73: 1, 74: 1, 75: 1, 76: 1, 77: 95, 78: 98, 79: 8, 80: 76, 81: 3, 82: 74, 83: 1, 84: 1, 85: 1, 86: 2, 87: 820, 88: 13, 89: 1, 90: 213, 91: 2, 92: 838, 93: 102, 94: 82, 95: 11, 96: 4, 97: 22, 98: 7, 99: 3, 100: 149, 101: 160, 102: 166, 103: 15, 104: 5, 105: 454, 106: 127, 107: 9, 121: 2404, 109: 339, 1

ValueError: too many values to unpack (expected 2)

## Beschreibung4
- 聚焦在从Beschreibung属性中，提取出新的属性

In [4]:
# 提取Auftragsnummer和Beschreibung属性
ab  = lb_wsp_2014[['Auftragsnummer', 'Beschreibung']]

In [80]:
print(ab.head())

  Auftragsnummer                                       Beschreibung  \
0     WSAU200371                                         WI RAD 3C0   
1     WSAU200371                                         WI RAD 3C0   
2     WSAU200673           Karosserie Lackierung vorbereitet          
3     WSAU200673  Karosserie Lackierung vorbereitet Korossionschutz   
4     WSAU200673                Farbton mit Mischanlage aufbereitet   

                                       Beschreibung2 Beschreibung3  
0                                           WIE RADE          RADE  
1                                           WIE RADE          RADE  
2               KAROSSERIEN LACKIERUNGEN VORBEREITET   KAROSSERIEN  
3  KAROSSERIEN LACKIERUNGEN VORBEREITET KOROSSION...   KAROSSERIEN  
4             FARBTONE MIT MISCHANLAGEN AUFBEREITETE           MIT  


##### 方法一： 提取名词，用空格分开

In [ ]:
# clean the beschreibung attribute
ab['Beschreibung2'] = ab['Beschreibung'].map(bs_operate)

In [ ]:
# ！！！！！可以改一下bs_operate中的条件， 目前只是排除了IN类型的对象，可以往里面再添加，比如去掉动词之类的。

##### 方法二： 根据描述中，出现的频率来分，使用频率最高的词，做为代表
eg. WI RAD 3C0: WI 的概率是0.1， RAD的是0.3， 3C0的是0.2，那么就用RAD来代替这个属性


In [66]:
# clean the beschreibung attribute
ab['Beschreibung2'] = ab['Beschreibung'].map(bs_operate)

/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [68]:
ab['Beschreibung2'].to_csv('/tmp/tmp.csv', sep = ';', index = False)

In [72]:
fdic = FreDict('/tmp/tmp.csv', header = True, sep = ';')

In [76]:
ab['Beschreibung3'] = ab['Beschreibung2'].map(fdic.get_best)

/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [78]:
ab.head(20)

,Auftragsnummer,Beschreibung,Beschreibung2,Beschreibung3
0,WSAU200371,WI RAD 3C0,WIE RADE,RADE
1,WSAU200371,WI RAD 3C0,WIE RADE,RADE
2,WSAU200673,Karosserie Lackierung vorbereitet,KAROSSERIEN LACKIERUNGEN VORBEREITET,KAROSSERIEN
3,WSAU200673,Karosserie Lackierung vorbereitet Korossionschutz,KAROSSERIEN LACKIERUNGEN VORBEREITET KOROSSION...,KAROSSERIEN
4,WSAU200673,Farbton mit Mischanlage aufbereitet,FARBTONE MIT MISCHANLAGEN AUFBEREITETE,MIT
5,WSAU200673,Rückwandklappe oben lackiert,RÜCKWANDKLAPPEN OBEN LACKIERTE,LACKIERTE
6,WSAU200673,BASISLACK,BASISLACKE,BASISLACKE
7,WSAU202971,Freigabe:1410159401-02,FREIGABEN,FREIGABEN
8,WSAU202971,GFS/GEFÜHRTE FUNKTION .,GFSE GEFÜHRTE FUNKTIONEN,GFSE
9,WSAU202971,SCHALLSCHUTZWANNE AUS- U.EINGEBAUT,SCHALLSCHUTZWANNEN AUS EINGEBAUTE,AUS


## Check n-1 relation

In [187]:
def check_to_1(df, atta, attb):
    """
    input:
        df, Dataframe:
            the data frame
        atta, string:
            first attribute
        attb, string:
            second attribute
    output:
        out boolean:
            if for an a has more than one b, return True, otherwise return False
    """
    df = df[[atta, attb]]
    df.to_csv('/tmp/tmp.csv', sep = ';', index = False)
    abdict = ABDict('/tmp/tmp.csv', header = True, sep = ';')
    dic = abdict.dic
    flag = 0
    for i,j in dic.items():
        if len(j) > 1:
            print(str(i) + '\t' + str(j))
    #for i in dic.values():
    #    if len(i) > 1:
    #        #print(i)
    #        flag = 1
    #if flag == 1:
    #    return True
    #else:
    #    return False

In [186]:
check_to_1(lb_wsp_2014, 'Beschreibung', 'AW-Nr')

18835
6556
WI RAD 3C0	['']
Karosserie Lackierung vorbereitet       	['51017133.0', '51017163.0', '51017193.0', '51017195.0', '51017173.0', '51017153.0', '51017197.0', '51017143.0']
Karosserie Lackierung vorbereitet Korossionschutz	['51017195.0']
Farbton mit Mischanlage aufbereitet	['51799999.0', '']
Rückwandklappe oben lackiert	['55597953.0', '55597902.0']
BASISLACK	['']
Freigabe:1410159401-02	['99999999.0']
GFS/GEFÜHRTE FUNKTION .	['1500000.0']
SCHALLSCHUTZWANNE AUS- U.EINGEBAUT	['10821900.0', '10821901.0', '10821950.0']
REP.SATZ	['']
LADELUFTSYSTEM GEPRÜFT	['21010135.0', '21010100.0']
ABDECKUNG AUS- U.EINGEBAUT	['10831900.0', '10831904.0', '10831950.0', '85221900.0', '10831936.0', '10831933.0']
DRUCKDOSE F TURBOLADER GEPRÜFT	['21290175.0']
Druckdose f Turbolader ersetzt	['21295533.0', '21295545.0', '21295500.0']
BATTERIEKONSOLE AUS- U.EINGEBAUT	['50841960.0', '50841900.0', '50841950.0']
LUFTFILTER AUS- U.EINGEBAUT	['23251945.0', '23251930.0', '23251933.0', '24251904.0', '24251912.0',

In [178]:
check_to_1(lb_wsp_2014, 'Auftragsnummer', 'KM-Stand')

21630
15490


False

In [191]:
check_to_1(data, 'Auftragsnummer', 'Markencode')

KeyboardInterrupt: 

In [188]:
check_to_1(lb_wsp_2014, 'Auftragsnummer', 'Adressanredecode')

21630
13


In [149]:
check_to_1(lb_wsp_2014, 'Auftragsnummer', 'Fahrgestellnummer')

21630
10195


False

In [150]:
check_to_1(lb_wsp_2014, 'Auftragsnummer', 'Motorcode')

21630
580


False

In [151]:
check_to_1(lb_wsp_2014, 'Auftragsnummer', 'Fahrzeugmodellnummer')

21630
2677


False

In [152]:
check_to_1(lb_wsp_2014, 'Auftragsnummer', 'Modell')

21630
2823


False

In [153]:
check_to_1(lb_wsp_2014, 'Auftragsnummer', 'Typ')

21630
55


False

In [154]:
check_to_1(lb_wsp_2014, 'Auftragsnummer', 'Getriebeartcode')

21630
9


False

In [180]:
check_to_1(lb_wsp_2014, 'Auftragsnummer', 'Auftragsdatum')

21630
317


False

In [155]:
check_to_1(lb_wsp_2014, 'Fahrgestellnummer', 'Modell')

10195
2823


False

In [157]:
check_to_1(lb_wsp_2014, 'Fahrgestellnummer', 'Typ')

10195
55


False

In [158]:
check_to_1(lb_wsp_2014, 'Fahrgestellnummer', 'Getriebecode')

10195
1397


False

In [189]:
check_to_1(lb_wsp_2014, 'Fahrgestellnummer', 'Adressanredecode')

10195
13
WAUZZZ4G8EN192168	['', 'Firma']
TRUZZZ8P8A1002681	['Herr', 'Frau']
WVWZZZ1KZBW577719	['Frau', 'Firma']
WVWZZZ1KZBM693243	['Firma', 'Herr']
WVWZZZ1KZAP144139	['Herr', 'Firma']
	['Firma', 'Herr', 'AN', 'Frau', '', 'OHNE', 'FA_F']
WAUZZZ8RXBA070037	['Frau', 'Herr']
WVWZZZ3CZ9E009337	['Firma', 'Herr']
WAUZZZ8K39A155895	['Firma', 'Herr']
TMBJF73T9E9028805	['', 'Firma']
WAUZZZ8P7AA043519	['Herr', '']
TMBZZZAAZED623579	['', 'Herr']
WVGZZZ5NZ9W084981	['Firma', 'Herr']
WV1ZZZ2FZB7002911	['Firma', 'Herr']
WAUZZZ8E58A100115	['Herr', 'Frau']
TMBNJ65J7E5025298	['', 'Firma']
WAUZZZ8R5EA046250	['', 'Herr']
TMBJD7NE5E0133379	['', 'Firma']
WAUZZZ8P2BA067762	['Herr', '']
WVWZZZ7NZDV033638	['', 'Herr']
TMBJB73T0E9037551	['', 'Firma']
WAUZZZ8K2EN027800	['', 'Firma']
WAUZZZ8R9EA075251	['', 'Firma']
TMBHT61Z5D2091979	['FA_F', 'Firma']
WAUZZZ8K59A092265	['Firma', 'Herr']
WVWZZZAUZDW047781	['Firma', 'Frau']
WV2ZZZ2KZBX236718	['Firma', '', 'Frau']
TMBHE61Z0C2034985	['Firma', 'Herr']
WAUZZZ4G1EN104058	

In [160]:
check_to_1(lb_wsp_2014, 'Fahrgestellnummer', 'Motorcode')

10195
580


False

In [179]:
check_to_1(lb_wsp_2014, 'Fahrgestellnummer', 'KM-Stand')

10195
15490
['82645,00', '87477,00', '92009,00', '95337,00']
['12537,00', '17228,00', '55598,00']
['75961,00', '70020,00']
['1013,00', '15,00']
['167164,00', '149947,00', '166000,00', '167656,00', '167653,00', '180658,00']
['64311,00', '58695,00', '62310,00', '62731,00', '64535,00', '67194,00', '91324,00']
['27132,00', '21945,00', '30177,00']
['124198,00', '124209,00']
['38110,00', '54501,00', '59806,00', '60684,00', '60687,00', '90691,00', '96374,00']
['91895,00', '92206,00']
['122325,00', '124833,00', '124846,00', '131361,00']
['112386,00', '118198,00', '131044,00']
['42783,00', '46350,00', '51816,00', '58370,00', '66609,00', '66696,00']
['180999,00', '185052,00', '200889,00', '210747,00']
['17784,00', '30073,00', '30331,00']
['114063,00', '121621,00', '130725,00', '143644,00', '150322,00']
['144873,00', '149311,00', '164210,00', '179854,00', '191407,00', '191405,00', '198102,00', '204638,00', '208996,00']
['92630,00', '113164,00']
['41438,00', '48214,00']
['79620,00', '89917,00', '9

True

## Markencode

In [166]:
mc = lb_wsp_2014['Markencode']

In [168]:
fake = mc.map(lambda x: bool(re.search('\d', str(x))))

In [173]:
#mc[fake]

In [171]:
mc.unique()

array(['VW', 'SKODA', 'AUDI', nan, 'BMW', '27.04.2006', 'OPEL', 'SUBARU',
       'SEAT', 'RENAULT', 'TOYOTA', '13.08.2009', 'FORD', 'CITROËN',
       'MERCEDES', 'MAZDA', 'PORSCHE', 'CHRYSLER', 'PEUGEOT', '09.05.2005'], dtype=object)

In [176]:
ar = lb_wsp_2014['Adressanredecode']

In [177]:
ar.unique()

array(['Firma', 'Herr', 'Frau', nan, 'AN', 'FA_F', 'FA_M', 'HR.DR.',
       'OHNE', 'AUTOHAUS', 'FR.DR.', 'HERR+FRAU', 'FAHRSCHULE'], dtype=object)

In [201]:
ab = lb_wsp_2014['Teile-Nr']

In [203]:
ab.describe()

count          127694
unique          15977
top       N  90813202
freq             3626
Name: Teile-Nr, dtype: object

In [207]:
ab.value_counts()

N  90813202       3626
G  052164DS       2448
B  000DS          2268
OEL02             1953
N  10648301       1623
03L115562         1533
1K1819653B        1436
1100              1063
G  013DS          1016
000010006         1013
N  0138157         881
WHT000729A         844
SKN  90813202      810
W-AG-EM            802
SKBV18             749
LMW030000Z0        748
8Z0827439B         704
1K0825951          692
E2000              643
2000               579
071115562C         574
D  00950025        515
ZVWLWASSER         509
W-LRA              489
H-S                482
03C115561H         473
311601361          472
AULGEWI            458
839601361          458
GEWI               458
                  ... 
1K5949101            1
N  0139594           1
8E0963557AH          1
03C103474J           1
2E0145856E           1
06B103217AC          1
1K0121447CT          1
5N0945106            1
SK04L145762AD        1
N  90766703          1
1K4831300G           1
SK03F103464D         1
SK000979133

In [211]:
ab = lb_wsp_2014[lb_wsp_2014['Teile-Nr'] == 'B  000DS']

In [212]:
ab = ab[['Teile-Nr', 'Fahrgestellnummer']]

In [213]:
ab['count'] = 1

In [216]:
bd = ab.groupby('Fahrgestellnummer', as_index = False).count().sort_values(by = 'count', ascending = False)

In [217]:
bd.reset_index()

,index,Fahrgestellnummer,Teile-Nr,count
0,749,WV2ZZZ2KZ9X011023,6,6
1,1135,WVWZZZ1JZXD209004,6,6
2,725,WV1ZZZ7HZCH010962,5,5
3,82,WAUZZZ4F3BN050880,5,5
4,1812,WVWZZZ6RZBY328496,5,5
5,1830,WVWZZZ6XZXW003891,5,5
6,1688,WVWZZZ3CZCE057210,4,4
7,1166,WVWZZZ1KZ5W522111,4,4
8,665,WV1ZZZ2KZ8X038558,4,4
9,1071,WVWZZZ1FZ7V000998,4,4
